In [1]:
using Pkg;Pkg.activate(".")
#Pkg.add.(["OnlineStats", "VegaDatasets", "DataFrames"])
#Pkg.add(path="../")

 Activating environment at `~/Documents/juliaPkgs/JSONLines/notebooks/Project.toml`


In [41]:
using JSONLines, OnlineStats, VegaDatasets, DataFrames, Statistics

# Get some JSONLines data

In [3]:
flights = dataset("flights-200k");

In [4]:
writelines("data/flights-200k.jsonl", flights)

# Read back in

In [5]:
flightsind = LineIndex("data/flights-200k.jsonl", nworkers = 4);

In [6]:
@assert flightsind |> Tables.rowtable == flights |> Tables.rowtable

# JSONLines functions

In [7]:
Base.displaysize() = (11, 80)

In [8]:
materialize(flightsind)

231083-element Array{NamedTuple{(:distance, :time, :delay),T} where T<:Tuple,1}:
 (distance = 405, time = 0.016666666666666666, delay = 14)
 (distance = 370, time = 5.5, delay = -11)
 (distance = 389, time = 5.666666666666667, delay = 5)
 ⋮
 (distance = 443, time = 22.616666666666667, delay = 106)
 (distance = 880, time = 22.8, delay = 125)
 (distance = 303, time = 22.95, delay = 29)

In [9]:
materialize(flightsind, row -> row[:distance]; eltype = Int)

231083-element Array{Int64,1}:
 405
 370
 389
   ⋮
 443
 880
 303

In [10]:
materialize(flightsind, row -> (;zip((:distance, :time), (row[:distance], row[:time]))...); eltype = NamedTuple{(:distance, :time), Tuple{Int, Float64}})

231083-element Array{NamedTuple{(:distance, :time),Tuple{Int64,Float64}},1}:
 (distance = 405, time = 0.016666666666666666)
 (distance = 370, time = 5.5)
 (distance = 389, time = 5.666666666666667)
 ⋮
 (distance = 443, time = 22.616666666666667)
 (distance = 880, time = 22.8)
 (distance = 303, time = 22.95)

In [11]:
filter(row -> row[:distance] > 400, flightsind)

102497-element Array{JSON3.Object{SubArray{UInt8,1,Array{UInt8,1},Tuple{UnitRange{Int64}},true},Array{UInt64,1}},1}:
 {
   "distance": 405,
       "time": 0.016666666666666666,
      "delay": 14
}
 {
   "distance": 405,
       "time": 6.166666666666667,
      "delay": -4
}
 {
   "distance": 562,
       "time": 6.25,
      "delay": 5
}
 ⋮
 {
   "distance": 423,
       "time": 22.166666666666668,
      "delay": -7
}
 {
   "distance": 443,
       "time": 22.616666666666667,
      "delay": 106
}
 {
   "distance": 880,
       "time": 22.8,
      "delay": 125
}

In [27]:
flightscol = columnwise(flightsind)

(delay = [14, -11, 5, -5, 3, 5, -4, -2, 0, 0  …  153, -5, 70, -16, -7, -7, -8, 106, 125, 29], distance = [405, 370, 389, 337, 303, 236, 405, 188, 197, 399  …  342, 588, 550, 550, 303, 423, 303, 443, 880, 303], time = [0.016666666666666666, 5.5, 5.666666666666667, 6.0, 6.0, 6.083333333333333, 6.166666666666667, 6.25, 6.25, 6.25  …  21.583333333333332, 21.75, 21.85, 21.916666666666668, 22.0, 22.166666666666668, 22.5, 22.616666666666667, 22.8, 22.95])

In [28]:
flightscol |> DataFrame!

,delay,distance,time
,Int64,Int64,Float64
1,14,405,0.0166667
2,-11,370,5.5
3,5,389,5.66667
4,-5,337,6.0
5,3,303,6.0
6,5,236,6.08333
7,-4,405,6.16667
8,-2,188,6.25
9,0,197,6.25


In [35]:
s = Series(Mean(), Variance());

In [36]:
for row in flightsind
    fit!(s, row[:distance])
end

In [37]:
s

Series
├─ Mean: n=231083 | value=506.803
└─ Variance: n=231083 | value=130055.0


In [38]:
c = CovMatrix();

In [42]:
var(flightscol.distance)

130055.360123533

In [43]:
mean(flightscol.distance)

506.8025081896981

In [39]:
for row in flightsind
    fit!(c, zip(row[:distance], row[:delay]))
end

In [40]:
c

CovMatrix: n=231083 | value=[130055.0 -590.544; -590.544 608.778]